This notebook aims to measure a variety of expectation values of the Majumdar Gosh Model groundstate using classical shadows (comparing pennylane and qiskit shadows).

In [1]:
import math
from qiskit import QuantumCircuit, transpile, Aer
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info.operators import Operator, Pauli, SparsePauliOp

import pennylane as qml

import matplotlib.pyplot as plt
import time

from concurrent.futures import ProcessPoolExecutor
import itertools

import operator
import numpy as np
np.random.seed(666)
np.set_printoptions(precision=3, suppress=True, linewidth=150)

In [2]:
import QiskitClassicalShadows as qcs
import PennylaneClassicalShadows as pcs

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator

In [4]:
#This function creates a groundstate Majumdar Gosh Model to test the Classical Shadow Reconstruction of States
system_size = 8

#This creates a circuit where the qubits are in the form |\psi> = 1/sqrt(2)(|01>-|10>)...1/sqrt(2)(|01>-|10>)
Q_MG_circuit = QuantumCircuit(system_size)
for i in range(0, system_size, 2): #Was previously for i in range(0, 10, 2)
            Q_MG_circuit.h(i)
            Q_MG_circuit.x(i+1)
            Q_MG_circuit.z(i)
            Q_MG_circuit.cnot(i,i+1)
            Q_MG_circuit.x(i)
            Q_MG_circuit.x(i+1)

num_snapshots = 50000
params = []

#calculate the shadow of the majumdar gosh circuit, shadow is a tuple containing array of measurement results
#and array of the unitary gates that rotated each qubit before measurement
Q_MG_shadow = qcs.calc_shadow(Q_MG_circuit, num_snapshots, system_size)

print("Qiskit Majumdar Gosh Model Classical Shadow size 50,000")
print("Here we have the results of the measurements of the qubits in the computational basis:")
print(Q_MG_shadow[0])
print("Here we have the unitary operator the acted upon each respective qubit(X:0,Y:1,Z:2):")
print(Q_MG_shadow[1])

Qiskit Majumdar Gosh Model Classical Shadow size 50,000
Here we have the results of the measurements of the qubits in the computational basis:
[[-1  1 -1 ... -1  1 -1]
 [-1  1 -1 ... -1  1  1]
 [ 1 -1  1 ...  1 -1  1]
 ...
 [ 1  1 -1 ... -1  1  1]
 [ 1  1  1 ... -1  1 -1]
 [ 1 -1  1 ... -1 -1  1]]
Here we have the unitary operator the acted upon each respective qubit(X:0,Y:1,Z:2):
[[0 2 1 ... 2 1 2]
 [0 1 2 ... 2 1 0]
 [2 0 1 ... 1 1 1]
 ...
 [0 2 0 ... 1 0 1]
 [1 0 0 ... 0 2 2]
 [0 0 1 ... 1 1 1]]


In [5]:
Q_MG_Observables = ["0X1X", "2Z6Z", "2X3X", "4Y5Y", "0X1X2X3X", "2Z3Z4X6X", "1X2X6Z7Z", "1X5Y", "4X5X", "4X5X6Z7Z"]
print(Q_MG_Observables)

['0X1X', '2Z6Z', '2X3X', '4Y5Y', '0X1X2X3X', '2Z3Z4X6X', '1X2X6Z7Z', '1X5Y', '4X5X', '4X5X6Z7Z']


In [6]:
#These are the analytic expectation values we expect for the above observables
analytic_expectation_values = [-1, 0, -1, -1, 1, 0, 0, 0, -1, 1]
print (analytic_expectation_values)

[-1, 0, -1, -1, 1, 0, 0, 0, -1, 1]


In [7]:
#Exact Expectation values calculated through cloud qasm simulator
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibmq_qasm_simulator")

estimator = Estimator(backend=backend)

Q_expval_exact = []

Q_exac_circ = Q_MG_circuit.copy()
for o in Q_MG_Observables:
    Q_op = qcs.obs_to_op(o, Q_exac_circ.num_qubits)
    Q_expectation_value = estimator.run(Q_exac_circ, Q_op).result().values
    Q_expval_exact.append(Q_expectation_value[0])

print(Q_expval_exact)

[-1.0, -0.0075, -1.0, -1.0, 1.0, 0.0155, -0.0105, 0.0095, -1.0, 1.0]


In [8]:
#Exact Expectation values calculated using local AER Simulator for those without an IBMQ account
from qiskit_aer.primitives import Estimator
estimator = Estimator(run_options= {"method": "statevector"})

Q_expval_exact = []
Q_exac_circ = Q_MG_circuit.copy()

for o in Q_MG_Observables:
    Q_op = qcs.obs_to_op(o, Q_exac_circ.num_qubits)
    Q_expectation_value = estimator.run(Q_exac_circ, Q_op).result().values
    Q_expval_exact.append(Q_expectation_value[0])
    
print(Q_expval_exact)

[-1.0, -0.013671875, -1.0, -1.0, 1.0, 0.009765625, -0.03515625, -0.0234375, -1.0, 1.0]


In [9]:
#These are the estimated expectation values using classical shadows for the above observables
Q_MG_exp_vals = [qcs.estimate_obervable(Q_MG_shadow, o) for o in Q_MG_Observables]
print(np.array(Q_MG_exp_vals))

[-1.     0.018 -1.    -1.     1.    -0.046  0.015 -0.007 -1.     1.   ]


In [10]:
#This function creates a groundstate Majumdar Gosh Model and calculates its classical shadow with size 10,000
system_size = 8
dev = qml.device("default.qubit", wires = system_size, shots = 1)

@qml.qnode(dev)
def P_MG_Circuit(params, **kawargs):
    observables = kawargs.pop("observables")
    for i in range(0, system_size, 2):
        qml.Hadamard(wires = i)
        qml.PauliX(wires = i+1)
        qml.PauliZ(wires = i)
        qml.CNOT(wires = [i,i+1])
        qml.PauliX(wires = i)
        qml.PauliX(wires = i+1)
    return [qml.expval(o) for o in observables]

num_snapshots = 50000
params = [] 

P_MG_shadow = pcs.calculate_classical_shadow(P_MG_Circuit, params, num_snapshots, system_size)

print("Pennylane Majumdar Gosh Model Classical Shadow size 50,000")
print("Here we have the results of the measurements of the qubits in the computational basis:")
print(P_MG_shadow[0])
print("Here we have the unitary operator the acted upon each respective qubit(X:0,Y:1,Z:2):")
print(P_MG_shadow[1])

Pennylane Majumdar Gosh Model Classical Shadow size 50,000
Here we have the results of the measurements of the qubits in the computational basis:
[[ 1.  1. -1. ... -1. -1.  1.]
 [-1. -1.  1. ...  1. -1.  1.]
 [-1.  1. -1. ... -1.  1. -1.]
 ...
 [ 1. -1.  1. ...  1. -1.  1.]
 [-1.  1. -1. ...  1. -1.  1.]
 [-1.  1. -1. ... -1. -1.  1.]]
Here we have the unitary operator the acted upon each respective qubit(X:0,Y:1,Z:2):
[[1 2 1 ... 2 0 1]
 [0 2 2 ... 2 2 0]
 [2 1 0 ... 1 1 1]
 ...
 [1 1 1 ... 0 0 0]
 [0 2 2 ... 1 2 1]
 [2 2 2 ... 2 0 1]]


In [11]:
P_MG_Observables = [qml.PauliX(0)@qml.PauliX(1), qml.PauliZ(2)@qml.PauliZ(6),qml.PauliX(2)@qml.PauliX(3), qml.PauliY(4)@qml.PauliY(5)
                 ,qml.PauliX(0)@qml.PauliX(1)@qml.PauliX(2)@qml.PauliX(3), qml.PauliZ(2)@qml.PauliZ(3)@qml.PauliX(4)@qml.PauliX(6)
                 ,qml.PauliX(1)@qml.PauliX(2)@qml.PauliZ(6)@qml.PauliZ(7), qml.PauliX(1)@qml.PauliY(5),qml.PauliX(4)@qml.PauliX(5),
                 qml.PauliX(4)@qml.PauliX(5)@qml.PauliZ(6)@qml.PauliZ(7)]

In [12]:
#These are the estimated expectation values using classical shadows for the above observables
P_MG_exp_vals = [pcs.estimate_shadow_obervable(P_MG_shadow, o) for o in P_MG_Observables]
print(np.array(P_MG_exp_vals))

[-1.    -0.019 -1.    -1.     1.    -0.024 -0.007 -0.005 -1.     1.   ]


The following lines of code test the qiskit majumdar gosh circuit created above with a sample set of rotations:
[2,2,2,1,1,0,2,0]

In [13]:
map_result = {"0": 1, "1": -1}
system_size = 8
MG_test = QuantumCircuit(system_size)
for i in range(0, system_size, 2): #Was previously for i in range(0, 10, 2)
    MG_test.h(i)
    MG_test.x(i+1)
    MG_test.z(i)
    MG_test.cnot(i,i+1)
    MG_test.x(i)
    MG_test.x(i+1)

MG_test.id(0)
MG_test.id(1)
MG_test.id(2)
#MG_test.sdg(3)
MG_test.rz(-np.pi/2, 3)
MG_test.h(3)
#MG_test.sdg(4)
MG_test.rz(-np.pi/2, 4)
MG_test.h(4)
MG_test.h(5)
MG_test.id(6)
MG_test.h(7)

MG_test.measure_all()
                    
simulator = Aer.get_backend('aer_simulator')

for i in range(10):
    compiled_circuit = transpile(MG_test, simulator)
    job = simulator.run(compiled_circuit, shots=1)
    result = job.result().get_counts(compiled_circuit)
    vals = [map_result[r] for r in list(result)[0][::-1]]
    print(vals)

[1, -1, 1, -1, -1, -1, 1, -1]
[1, -1, 1, -1, -1, 1, 1, -1]
[-1, 1, -1, -1, -1, 1, -1, -1]
[1, -1, -1, -1, -1, 1, 1, 1]
[1, -1, 1, -1, -1, -1, 1, -1]
[-1, 1, 1, -1, -1, -1, 1, -1]
[1, -1, -1, -1, 1, -1, 1, 1]
[-1, 1, 1, 1, -1, -1, -1, -1]
[1, -1, -1, -1, 1, -1, 1, -1]
[-1, 1, 1, -1, 1, 1, -1, 1]


The following lines of code test the shadow created for the qiskit MG circuit

In [14]:
vals = Q_MG_shadow[0]
count1 = 0
count2 = 0
for v in vals:
    count1 = count1 + 1
    for v2 in v:
        count2 = count2 + 1
        if count1 == 500:
            print(v)
        if not(v2 in [1,-1]):
            print("Error")
print(count1)
print(count2)
unitaries = Q_MG_shadow[1]
count3 = 0
count4 = 0
for unitary in unitaries:
    count3 = count3 + 1
    for u in unitary:
        count4 = count4 + 1
        if count3 == 500:
            print(unitary)
        if not(u in [0,1,2]):
            print("Error")
print((Q_MG_shadow[1])[30000])
print(count3)
print(count4)

[ 1 -1 -1  1 -1  1  1 -1]
[ 1 -1 -1  1 -1  1  1 -1]
[ 1 -1 -1  1 -1  1  1 -1]
[ 1 -1 -1  1 -1  1  1 -1]
[ 1 -1 -1  1 -1  1  1 -1]
[ 1 -1 -1  1 -1  1  1 -1]
[ 1 -1 -1  1 -1  1  1 -1]
[ 1 -1 -1  1 -1  1  1 -1]
50000
400000
[2 2 2 1 1 0 2 0]
[2 2 2 1 1 0 2 0]
[2 2 2 1 1 0 2 0]
[2 2 2 1 1 0 2 0]
[2 2 2 1 1 0 2 0]
[2 2 2 1 1 0 2 0]
[2 2 2 1 1 0 2 0]
[2 2 2 1 1 0 2 0]
[1 0 1 2 1 2 1 0]
50000
400000
